# *Libra* - Declaring Models in in-line Code
> Brady Spears, Los Alamos National Laboratory

---

## License Information
Copyright 2024

Triad National Security, LLC. All rights reserved.
This program was produced under U.S. Government contract 89233218CNA000001 for 
Los Alamos National Laboratory (LANL), which is operated by Triad National 
Security, LLC for the U.S. Department of Energy/National Nuclear Security 
Administration. All rights in the program are reserved by Triad National 
Security, LLC, and the U.S. Department of Energy/National Nuclear Security 
Administration. The Government is granted for itself and others acting on its 
behalf a nonexclusive, paid-up, irrevocable worldwide license in this material 
to reproduce, prepare derivative works, distribute copies to the public, 
perform publicly and display publicly, and to permit others to do so.
 
Permission is hereby granted, free of charge, to any person obtaining a copy of 
this software and associated documentation files (the "Software"), to deal in 
the Software without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies 
of the Software, and to permit persons to whom the Software is furnished to do 
so, subject to the following conditions:
 
The above copyright notice and this permission notice shall be included in all 
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR 
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, 
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE 
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER 
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, 
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE 
SOFTWARE.

---

## About *Libra*
`Libra` is an open-source Python package for managing object-relation-mapped 
(ORM) instances (A.K.A. "Models") across various schemas and various relational 
database backends. `Libra` was developed by Brady Spears with 
contirubtions from Christine Gammans and Richard Alfaro-Diaz at Los Alamos 
National Laboratory.

---

## About this Notebook
This notebook demonstrates various acceptable syntax when declaring models, 
behavior when manipulating those models, and optional functionality to augment 
model behavior.

---

## Model Declaration Syntax Option #1
(First model declaration lifted directly from [/docs/examples/libra_intro.ipynb](libra_intro.ipynb))

In [ ]:
from datetime import datetime, timezone

from sqlalchemy import Column
from sqlalchemy import create_engine
from sqlalchemy import DateTime, Float, Integer, String
from sqlalchemy.orm import Session

from libra import Schema

# Initialization only needs the Schema name
kbcore = Schema('NNSA KB Core')

@kbcore.add_model
class affiliation:
    net     = Column(String(8), default = '-')
    sta     = Column(String(6), default = '-')
    time    = Column(Float(precision = 53), default = -9999999999.999)
    endtime = Column(Float(precision = 53), default = 9999999999.999)
    lddate  = Column(DateTime, nullable = False, onupdate = datetime.now(timezone.utc),
        default = datetime.now(timezone.utc))

    pk = ['net', 'sta', 'time']

@kbcore.add_model
class site:
    sta = Column(String(6), default = '-')
    ondate = Column(Integer, default = -1)
    offdate = Column(Integer, default = 2286324)
    lat = Column(Float(precision = 53), default = -999.)
    lon = Column(Float(precision = 53), default = -999.)
    elev = Column(Float(precision = 24), default = -999.)
    staname = Column(String(50), default = '-')
    statype = Column(String(4), default = '-')
    refsta = Column(String(6), default = '-')
    dnorth = Column(Float(precision = 24))
    deast = Column(Float(precision = 24))
    lddate = Column(DateTime, nullable = False, onupdate = datetime.now(timezone.utc),
        default = datetime.now(timezone.utc))

    pk = ['ondate', 'sta']

engine = create_engine('sqlite:///data/libra_declaremodels.db')
session = Session(engine)

class Affiliation(kbcore.affiliation):
    __tablename__ = 'decmod_affiliation'

class Site(kbcore.site):
    __tablename__ = 'decmod_site'

Affiliation.__table__.drop(engine, checkfirst = True) # Drop table if already there
Site.__table__.drop(engine, checkfirst = True)

kbcore.base.metadata.create_all(engine) # Create Tables

# Add data
session.add(Affiliation('USGS', 'ANMO', 144547200, None, None))
session.add(Site(sta = 'ANMO', ondate = 2002324, lat = 34.94591, lon = -106.4572, 
    elev = 1.82, staname = 'Albuquerque, New Mexico', statype = 'ss', 
    refsta = 'ANMO', dnorth = 0., deast = 0.))

session.commit()

# Pull it back out
print(list(session.query(Site).all()))
print(list(session.query(Affiliation).all()))

[Site(ondate=2002324, sta='ANMO')]
[Affiliation(net='USGS', sta='ANMO', time=144547200.0)]


## Model Declaration Option #2

In [28]:
from datetime import datetime, timezone

from sqlalchemy import Column
from sqlalchemy import DateTime, Float, Integer, String
from sqlalchemy.orm import declared_attr
from sqlalchemy import PrimaryKeyConstraint

from libra import Schema

# ==============================================================================

css = Schema('CSS 3.0')

# Use traditional SQLAlchemy Declarative Syntax to add models to a Schema
@css.add_model
class event:
    evid = Column(Integer, default = '-')
    evname = Column(String(15), default = '-')
    prefor = Column(Integer, default = '-')
    auth = Column(String(15), default = '-')
    commid = Column(Integer, default = -1)
    lddate = Column(DateTime, nullable = False, onupdate = datetime.now(timezone.utc),
        default = datetime.now(timezone.utc))
    
    @declared_attr
    def __table_args__(cls):
        return (PrimaryKeyConstraint('evid'), )

# ==============================================================================

connection_str = 'sqlite:///data/libra_declaremodels.db' # In-Memory DB

engine = create_engine(connection_str)
session = Session(engine)

class Event(css.event):
    __tablename__ = 'decmod_event'

Event.__table__.drop(engine, checkfirst = True) # Drop table if already there
css.base.metadata.create_all(engine) # Create Table

# Add Data to the table
for i in range(0,10):
    session.add(Event(evid = i, evname = f'Seismic Event #{i}', prefor = i, auth = 'Me'))
session.commit()

# Pull it back out
print(list(session.query(Event).all()))

[Event(evid=0), Event(evid=1), Event(evid=2), Event(evid=3), Event(evid=4), Event(evid=5), Event(evid=6), Event(evid=7), Event(evid=8), Event(evid=9)]


---

## SQLAlchemy Type Override

`SQLAlchemy` makes use of custom `Type` objects 
(https://docs.sqlalchemy.org/en/20/core/type_basics.html#module-sqlalchemy.types), 
which, depending on the database backend being used, will force the Data 
Definition Language (DDL) to translate that any model declaration of that 
object into the appropriate, dialect-specific SQL type. `SQLAlchemy` does a 
tremendous job of mapping dialect-specific types to their "Camel-Case" `Type` 
objects, but sometimes, the generic types in the object-oriented environment 
are not exactly mapped to what is desired in the relational database environment. 

`Libra` uses `SQLAlchemy`'s flexibility to overcome this issue by extending 
support through the `TypeMap` object, which acts as a hash map for various 
built-in or derivative types. A `TypeMap` object is attached to a schema and 
forces all models belonging to that schema to replace the specific type with 
the mapped type.

One real-world use-case for this is Oracle Database's handling of string-like 
types. When using `SQLALchemy`'s `String(30)` object, the DDL automatically 
translates this as "VARCHAR2(30 CHAR)". For some use cases, especially on legacy 
systems, an different, byte-encoded "VARCHAR2(30 BYTE)" is desired.

In [ ]:
from datetime import datetime, timezone

from sqlalchemy import Column
from sqlalchemy import DateTime, Float, Integer, String
from sqlalchemy.orm import declared_attr
from sqlalchemy.ext.compiler import compiles

from libra import Schema
from libra.util import TypeMap

# ==============================================================================
# Define a custom byte-encoded VARCHAR2 type

class VARCHAR2Byte(String):
    ...

@compiles(VARCHAR2Byte)
def _compile(_type, compiler, **kw):
    len = _type.length

    return 'VARCHAR(%i BYTE)' % len # This string returned to the DDL

# ==============================================================================
# Declare Model & Schema using custom TypeMap

typemap = TypeMap({'String' : VARCHAR2Byte}) # Maps String type to VARCHAR2Byte

css27 = Schema('CSS 2.7', typemap = typemap) # Inject into schema

@css27.add_model
class code:
    code   = Column(String(6))
    rel    = Column(String(7))
    attrib = Column(String(7))
    coddes = Column(String(255))

    pk = ['attrib', 'code', 'rel']

class Code(css27.code):
    __tablename__ = 'decmod_code'

print(Code.attrib.type) # String(7) now maps to VARCHAR(7 BYTE)

VARCHAR(7 BYTE)
